## Многоагентная реализация разговорного агента

In [1]:
sentences = [
    "Расскажи всё, что ты знаешь про сорт вина мерло",
    "Какие существуют сорта винограда для изготовления красного вина?",
    "Какие вина подойдут к итальянской пасте?"
]

In [14]:
def mkstate(text):
    return { 'input' : text }

In [19]:
import os
from yandex_chain import YandexGPTClassifier

auth = {
    "folder_id" : os.environ['folder_id'].strip(), 
    "api_key" : os.environ['api_key'].strip()
}

class Agent:
    def __call__(self, state):
        return "new_state"
    
class Classfier(Agent):
    def __init__(self, task_description, labels, samples=None):
        self.classifier = YandexGPTClassifier(task_description, labels, samples, **auth)

    def __call__(self, state):
        res = self.classifier.invoke(state['input'])
        c = self.classifier.get_top_label(res)
        return c
    
InputClassifier = Classfier(
    """Определи, содержится ли в вопросе одна из следующих задач:
* подобрать вино к еде (подбор_вина),
* подобрать еду к вину (подбор_еды),
* другой вопрос (другая_тема)""",
    ["подбор_вина","подбор_еды","другая_тема"]
)

for s in sentences:
    res = InputClassifier(mkstate(s))
    print(f"{s} -> {res}")

Расскажи всё, что ты знаешь про сорт вина мерло -> другая_тема
Какие существуют сорта винограда для изготовления красного вина? -> подбор_вина
Какие вина подойдут к итальянской пасте? -> подбор_вина


In [28]:
from yandex_chain import YandexLLM
import json

class NER(Agent):
    def __init__(self, prompt, on_success, on_fail, labels=None):
        self.llm = YandexLLM(instruction_text=prompt, **auth)
        self.labels = labels
        self.on_success = on_success
        self.on_fail = on_fail

    def __call__(self, state):
        res = self.llm.invoke(state['input'])
        if res=="NONE" or res=='(NONE)':
            return self.on_fail
        if '('in res: res = res[res.find('(')+1:]
        if ')' in res: res = res[res.find(')')]
        res = [x.strip() for x in res.split('|')]
        if self.labels:
            res = [ x for x in res if x in self.labels and x!='NONE']
        if len(res)==0:
            return self.on_fail
        state['entities'] = res
        return self.on_success

FoodNER = NER(
    """Твоя задача - выделить из запроса название упомянутых там блюд, из приведённого ниже в тройных обратный кавычках списка:
```
PASTA - итальянская паста или макароны
FISH - рыба
MEAT - мясо, говядина или свинина, стейк
```
В качестве результата выведи только список сущностей из упомянутого списка (PASTA, FISH, MEAT), в круглых скобках через знак |, например (FISH|MEAT). Если в запросе не содержится упоминания ни об одной из сущностей, выведи NONE. Не выводи ничего кроме результирующего списка.
""",
"найти_вино","другая_тема",
["PASTA","MEAT","FISH"]
)

for s in sentences:
    x = mkstate(s)
    res = FoodNER(x)
    print(f"{s} -> {res}")

Расскажи всё, что ты знаешь про сорт вина мерло -> другая_тема
Какие существуют сорта винограда для изготовления красного вина? -> другая_тема
PASTA | NONE
['PASTA', 'NONE']
Какие вина подойдут к итальянской пасте? -> найти_вино


In [42]:
with open('../graph_rag/graphs/entities.json',encoding='utf-8') as f:
    entities = json.load(f)
with open('../graph_rag/graphs/relations.json',encoding='utf-8') as f:
    relations = json.load(f)

GeneralNER = NER(
"""
В запросе приводится короткий текст. Тебе необходимо выделить из него все сущности,
похожие на сущности из списка в двойных кавычках: "{list}". Верни только список сущностей в скобках
через знак |, например: (сира|ЮАР). Верни только те сущности, которые в явном виде
присутствуют в запросе. Не придумывай никакие дополнительные сущности и не рассуждай. Если сущностей
в тексте нет, верни NONE.
""".replace('{list}',', '.join(entities.keys())),
"graph_rag","simple_rag"
)


In [43]:
import networkx as nx

class GraphRAG(Agent):

    answer_prompt = """
Тебе задан следующий запрос от пользователя: {question}.
Ответь на этот вопрос, используя при этом информацию, содержащуюся ниже в тройных обратных кавычках:
```
{context}
```
"""

    def __init__(self,entities,relations,level=2):
        self.llm = YandexLLM(**auth)
        self.entities = entities
        self.relations = relations
        self.level = level

    def populate_graph(self,G,e,level=None):
        if e in G.nodes:
            return
        if e in self.entities.keys():
            G.add_node(e, label=e)
        if level is not None and level<=0:
            return
        new_ent = set(
            [r['source'] for r in relations if r['target'] == e] + 
            [r['target'] for r in relations if r['source'] == e])
        for ne in new_ent:
            self.populate_graph(G,ne,None if level is None else level-1)
        for r in relations:
            if r['source'] == e:
                G.add_edge(e, r['target'], label=r['relation'], desc=r['desc'])
            if r['target'] == e:
                G.add_edge(r['source'], e, label=r['relation'], desc=r['desc'])

    def __call__(self, state):
        G = nx.DiGraph()
        for x in state['entities']:
            self.populate_graph(G,x,self.level)
        ctx = '\n'.join(e[-1]['desc'] for e in G.edges(data=True))
        state['output'] = self.llm.invoke(self.answer_prompt
            .replace('{context}',ctx)
            .replace('{question}',state['input']))
        return 'конец'
    
TheGraphRAG = GraphRAG(entities,relations,2)


In [44]:
class AgentRuntime:
    def __init__(self,states):
        self.states = states
        
    def run(self, state, start_state, verbose=False):
        s = start_state
        while True:
            if verbose:
                print(f"Executing state: {s}, state map = {state}")
            A = self.states[s]
            s = A(state)
            if s == 'конец' or s is None:
                break

table = {
    'начало' : InputClassifier,
    'другая_тема' : GeneralNER,
    'graph_rag' : TheGraphRAG
}

AR = AgentRuntime(table)
AR.run(mkstate(sentences[0]), 'начало', verbose=True)

Executing state: начало, state map = {'input': 'Расскажи всё, что ты знаешь про сорт вина мерло'}
Executing state: другая_тема, state map = {'input': 'Расскажи всё, что ты знаешь про сорт вина мерло'}
Каберне фран, франция, Испания, зифандель, Чили, ЮАР, Австралия, белое вино, десертное вино, розовое вино, фруктовый профиль, бордо, Италия, примативо, Хорватия, Далмация, папринетто, тиховинная зона, сухое, полусухое, десертное, розанское, коллекционное, полнотелое, тёмно-рубиновый, сорт винограда, средиземноморский климат, глинисто-сланцевые почвы, рибейра де дуэро, биерсо, торонто, виноградники наэ.

**Сорт вина мерло**
Мерло — популярный благородный сорт красного вина, который часто входит в купаж известных вин. Большая часть вина Мерло производится во Франции, в Бордо. Кроме этого, его выращивают и делают из него вина в других странах: Италии, Чили, Аргентине, Испании, США и других. Практически всегда этот сорт используется для создания элитных красных вин, так как сам по себе облада

KeyError: 'entities'

In [45]:
entities.keys()

dict_keys(['каберне фран', 'франция', 'испания', 'зинфандель', 'сша', 'калифорния', 'италия', 'примитиво', 'хорватия', 'далмация', 'чили', 'юар', 'австралия', 'красное вино', 'белое вино', 'десертное вино', 'розовое вино', 'фруктовый профиль', 'сахар', 'американский национальный виноград', 'американская кухня', 'золота лихорадка', 'сухой закон', 'старатель', 'соус чили', 'чатни', 'манго', 'третье ноября', 'национальный день зинфанделя в сша', 'фан-клуб', 'zap', 'южная африка', 'новая зеландия', 'россия', 'бордо', 'крепкие', 'полнотелые', 'средней кислотностью', 'танины', 'низкокислотные', 'белки', 'жиры', 'блюда с высоким содержанием жиров и белков', 'паста болоньезе', 'перец чили', 'сыры', 'шоколад', 'грибы', 'болгарский перец', 'вина', 'напа', 'карменер', 'китай', 'мерло (фр. merlot)', 'красный', 'черный дрозд', 'виноградник', 'либурне', 'медок', 'каберне совиньон', 'урожайность', 'томаты', 'арабьята', 'лосось', 'тёмный оттенок', 'акцент', 'ваниль', 'говядина', 'десерт', 'европа', 'к